In [76]:
# Multiline string input
input_columns = """
DocumentNumber
PlantCode
MaterialNumber
AccountAssignment
SoldToCustomerCode
SalesEmployeeCode
Doctor1
Doctor2
ShipToCustomerCode
BillToCustomercode
SalesOrderType
StorageLocation
"""

key_columns_input = """
DocumentNumber
PlantCode
MaterialNumber
"""
table_name_input = "DimCountry"


column_names = [line.strip() for line in input_columns.strip().split('\n')]
key_columns = [line.strip() for line in key_columns_input.strip().split('\n')]

table_name_origin = f'[' + table_name_input + ']'
table_name_ssis = '[" + @[System::PackageName] + "]'

In [77]:
table_name = table_name_ssis
print(table_name)

[" + @[System::PackageName] + "]


In [ ]:
header = '___EDW___\n'
select_statement = 'SELECT\n'
select_statement += '[' + column_names[0] + ']\n'
for column in column_names[1:]:
    select_statement += f', [{column}]\n'
select_statement += 'FROM [DBO].'+ table_name_ssis + ' WITH (NOLOCK)'
print(header)
print(select_statement)


header = '\n\n___DIM___\n'
select_statement = 'SELECT\n'
select_statement += f'[' + column_names[0] + ']' + 'AS Destination' + column_names[0] +'\n'
for column in column_names[1:]:
    select_statement += f', [{column}] AS Destination{column}\n'
select_statement += ', [IsActive] AS DestinationIsActive\n'
select_statement += 'FROM [dbo].'+ table_name_ssis + ' WITH (NOLOCK)'
print(header)
print(select_statement)

header = '\n\n___UPDATE___\n'
update_statement = 'UPDATE\n[dbo].[' + table_name_origin + ']\nSET\n'
for column in column_names:
    update_statement += f'[{column}] = ?,\n'
update_statement += "[Source] = 'ST',\n[ModifiedBy] = 'system',\n[ModifiedDate] = GETDATE(),\n[IsActive] = 1\n"
update_statement += 'WHERE\n'
for key in key_columns:
    update_statement += f'[{key}] = ?\nAND '
update_statement = update_statement.rstrip('AND ') # remove the last AND
print(header)
print(update_statement)

header = '\n\n___DISABLE___\n'
disable_statement = 'UPDATE [dbo].' + table_name_origin
disable_statement += '\nSET IsActive = 0'
disable_statement += '\nWHERE'
for key in key_columns:
    disable_statement += f'[{key}] = ?\nAND '
disable_statement = disable_statement.rstrip('AND ')
print(header)
print(disable_statement)


In [79]:
def generate_isnull_expression(column_name):
    return f"(ISNULL([{column_name}]) ? \"\" : [{column_name}])"

# Split the input string into a list of column names
column_names = [line.strip() for line in input_columns.strip().split('\n')]

# Generate and print the ISNULL expressions for each column name
isnull_expressions = [generate_isnull_expression(column) for column in column_names]

# Print the results
for expression in isnull_expressions:
    print(expression)


(ISNULL([DocumentNumber]) ? "" : [DocumentNumber])
(ISNULL([PlantCode]) ? "" : [PlantCode])
(ISNULL([MaterialNumber]) ? "" : [MaterialNumber])
(ISNULL([AccountAssignment]) ? "" : [AccountAssignment])
(ISNULL([SoldToCustomerCode]) ? "" : [SoldToCustomerCode])
(ISNULL([SalesEmployeeCode]) ? "" : [SalesEmployeeCode])
(ISNULL([Doctor1]) ? "" : [Doctor1])
(ISNULL([Doctor2]) ? "" : [Doctor2])
(ISNULL([ShipToCustomerCode]) ? "" : [ShipToCustomerCode])
(ISNULL([BillToCustomercode]) ? "" : [BillToCustomercode])
(ISNULL([SalesOrderType]) ? "" : [SalesOrderType])
(ISNULL([StorageLocation]) ? "" : [StorageLocation])
